In [2]:
import cv2
import os

def video_to_frames(video_path, output_folder):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        frame_filename = os.path.join(output_folder, f"{frame_idx}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_idx += 1

    cap.release()
    print(f"Finished extracting {frame_idx} frames to '{output_folder}'.")

# Example usage
video_to_frames("data/test_video/zambrow_30m.mkv", "frames")
#video_to_frames("data/test_video/acc0.mkv", "acc0.mkv_")
#video_to_frames("data/test_video/acc1.mkv", "acc1.mkv_")
#video_to_frames("data/test_video/acc2.mkv", "acc2.mkv_")
#video_to_frames("data/test_video/acc3.mkv", "acc3.mkv_")
#video_to_frames("data/test_video/acc4.mkv", "acc4.mkv_")

Finished extracting 57571 frames to 'frames'.


In [4]:
import os
import shutil
from glob import glob
from natsort import natsorted

def collect_frames(folder):
    return natsorted(glob(os.path.join(folder, "*.jpg")))

def copy_sequence(frames, dest_folder, start_index, is_accident=False):
    for frame in frames:
        suffix = "" if is_accident else ""
        new_filename = os.path.join(dest_folder, f"{start_index}{suffix}.jpg")
        shutil.copy(frame, new_filename)
        start_index += 1
    return start_index

def mix_sequences(normal_folder, accident_folders, output_folder, interval=500):
    os.makedirs(output_folder, exist_ok=True)

    normal_frames = collect_frames(normal_folder)
    accident_frames_list = [collect_frames(f) for f in accident_folders]

    out_idx = 0
    normal_idx = 0
    acc_idx = 0

    while normal_idx < len(normal_frames):
        # Add normal frames
        end_idx = min(normal_idx + interval, len(normal_frames))
        out_idx = copy_sequence(normal_frames[normal_idx:end_idx], output_folder, out_idx, is_accident=False)
        normal_idx = end_idx

        # Add accident frames
        if acc_idx < len(accident_frames_list):
            out_idx = copy_sequence(accident_frames_list[acc_idx], output_folder, out_idx, is_accident=True)
            acc_idx += 1

    print(f"Mixed frames saved to '{output_folder}' with accident frames ending in 'a.jpg'.")

# Example usage
mix_sequences(
    normal_folder="data/test_frames/normal_zambrow",
    accident_folders=["data/test_frames/acc0", "data/test_frames/acc1", "data/test_frames/acc2", "data/test_frames/acc3", "data/test_frames/acc4"],
    output_folder="combined",
    interval=2000  # Adjust for how often you want to insert accidents
)

Mixed frames saved to 'combined' with accident frames ending in 'a.jpg'.


In [7]:
import os
import re

def remove_letters_before_extension(directory):
    for filename in os.listdir(directory):
        # Full path of the file
        old_path = os.path.join(directory, filename)
        if os.path.isfile(old_path):
            # Split base name and extension
            base, ext = os.path.splitext(filename)

            # Remove letters from the base name, keep numbers and other chars
            new_base = re.sub(r'[a-zA-Z]', '', base)

            # Build new filename
            new_filename = new_base + ext
            new_path = os.path.join(directory, new_filename)

            # Rename the file
            print(f'Renaming "{filename}" to "{new_filename}"')
            os.rename(old_path, new_path)

# Replace 'your_directory_path_here' with the path to your folder
remove_letters_before_extension('data/test/combined')

Renaming "0.jpg" to "0.jpg"
Renaming "1.jpg" to "1.jpg"
Renaming "10.jpg" to "10.jpg"
Renaming "100.jpg" to "100.jpg"
Renaming "1000.jpg" to "1000.jpg"
Renaming "10000.jpg" to "10000.jpg"
Renaming "10001.jpg" to "10001.jpg"
Renaming "10002.jpg" to "10002.jpg"
Renaming "10003.jpg" to "10003.jpg"
Renaming "10004.jpg" to "10004.jpg"
Renaming "10005.jpg" to "10005.jpg"
Renaming "10006.jpg" to "10006.jpg"
Renaming "10007.jpg" to "10007.jpg"
Renaming "10008.jpg" to "10008.jpg"
Renaming "10009.jpg" to "10009.jpg"
Renaming "1001.jpg" to "1001.jpg"
Renaming "10010.jpg" to "10010.jpg"
Renaming "10011.jpg" to "10011.jpg"
Renaming "10012.jpg" to "10012.jpg"
Renaming "10013.jpg" to "10013.jpg"
Renaming "10014.jpg" to "10014.jpg"
Renaming "10015.jpg" to "10015.jpg"
Renaming "10016.jpg" to "10016.jpg"
Renaming "10017.jpg" to "10017.jpg"
Renaming "10018.jpg" to "10018.jpg"
Renaming "10019.jpg" to "10019.jpg"
Renaming "1002.jpg" to "1002.jpg"
Renaming "10020.jpg" to "10020.jpg"
Renaming "10021.jpg" to 

In [2]:
import os
import shutil
import random
from glob import glob
from natsort import natsorted

def collect_frames(folder):
    return natsorted(glob(os.path.join(folder, "*.jpg")))

def copy_sequence(frames, dest_folder, start_index, is_accident=False):
    for frame in frames:
        suffix = "" if is_accident else ""
        new_filename = os.path.join(dest_folder, f"{start_index}{suffix}.jpg")
        shutil.copy(frame, new_filename)
        start_index += 1
    return start_index

def mix_sequences_random(normal_folder, accident_folders, output_folder, min_gap=1000):
    os.makedirs(output_folder, exist_ok=True)

    normal_frames = collect_frames(normal_folder)
    accident_frames_list = [collect_frames(f) for f in accident_folders]
    total_normal = len(normal_frames)
    accident_count = len(accident_frames_list)

    # Determine valid random insertion points with min_gap constraint
    insertion_points = []
    last_insertion = -min_gap
    while len(insertion_points) < accident_count:
        candidate = random.randint(last_insertion + min_gap, total_normal - 1)
        if candidate >= total_normal:
            break
        insertion_points.append(candidate)
        last_insertion = candidate

    # Sort insertion points so we can insert while walking through normal frames
    insertion_points = sorted(insertion_points)

    out_idx = 0
    normal_idx = 0
    acc_idx = 0

    for point in insertion_points:
        # Copy normal frames up to insertion point
        while normal_idx < min(point, total_normal):
            out_idx = copy_sequence([normal_frames[normal_idx]], output_folder, out_idx, is_accident=False)
            normal_idx += 1

        # Insert accident sequence
        if acc_idx < accident_count:
            out_idx = copy_sequence(accident_frames_list[acc_idx], output_folder, out_idx, is_accident=True)
            acc_idx += 1

    # Copy remaining normal frames
    while normal_idx < total_normal:
        out_idx = copy_sequence([normal_frames[normal_idx]], output_folder, out_idx, is_accident=False)
        normal_idx += 1

    print(f"Frames mixed with accident clips randomly inserted (min gap: {min_gap}) into '{output_folder}'.")

# Example usage
mix_sequences_random(
    normal_folder="data/test_frames/normal_zambrow",
    accident_folders=["data/test_frames/acc0", "data/test_frames/acc1", "data/test_frames/acc2", "data/test_frames/acc3", "data/test_frames/acc4"],
    output_folder="combined",
    min_gap=1000,
)

ValueError: empty range in randrange(58438, 57571)

In [1]:
import os
import shutil
from glob import glob
from natsort import natsorted

def collect_frames(folder):
    return natsorted(glob(os.path.join(folder, "*.jpg")))

def copy_sequence(frames, dest_folder, start_index):
    for frame in frames:
        new_filename = os.path.join(dest_folder, f"{start_index}.jpg")
        shutil.copy(frame, new_filename)
        start_index += 1
    return start_index

def combine_accidents_only(accident_folders, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    out_idx = 0

    for acc_folder in accident_folders:
        acc_frames = collect_frames(acc_folder)
        out_idx = copy_sequence(acc_frames, output_folder, out_idx)

    print(f"All accident sequences combined into '{output_folder}'.")

# Example usage
combine_accidents_only(
    accident_folders=[
        "data/test_frames/acc0",
        "data/test_frames/acc1",
        "data/test_frames/acc2",
        "data/test_frames/acc3",
        "data/test_frames/acc4"
    ],
    output_folder="combined_accidents_only"
)


All accident sequences combined into 'combined_accidents_only'.
